# Parse heat capacity data

In [10]:
import glob, os
import numpy as np
from scipy.interpolate import PchipInterpolator

from thermotools.plot import *
from thermotools import get_inpdir, get_gendir, empty_dir

tmin = 0.5
tmax = 4000.0
dt   = 0.1

empty_dir(os.path.join(get_gendir(), "cp", "dat"))
empty_dir(os.path.join(get_gendir(), "cp", "plt"))

In [11]:
def convert_name(name:str):
    # drop number from single-atoms in molecule
    gas = name[0]+""
    for i in range(1,len(name)):
        if (name[i] == "1") and (name[i-1].isalpha()):
            continue
        gas += name[i]
    return gas

In [12]:
def plot_gas(fpath:str):
    gas = fpath.split("/")[-1].split(".")[0]
    data = np.loadtxt(fpath, delimiter=',').T
    t = data[0]
    c = data[1]

    fig,ax = plt.subplots(1,1, figsize=(5,4))
    ax.plot(t,c, c='k')
    ax.set(xlabel="T [K]", ylabel=r"C$_{\text{p}}$ [J mol$^{-1}$ K$^{-1}$]")

    pltdir = os.path.join(get_gendir(), "cp", "plt")
    fig.savefig(pltdir+"/%s.png"%gas, bbox_inches='tight', dpi=170)
    plt.close("all")

In [13]:
def convert_txt(inpath:str):
    # get gas name
    gas = inpath.split("/")[-1].split(".")[0]
    gas = convert_name(gas)

    # load data from txt
    with open(inpath,'r') as hdl:
        lines = hdl.readlines()

    t_data = []
    c_data = []
    for i,line in enumerate(lines[3:]):
        # read t,c from row
        t = str(line.split("\t")[0]).strip()
        c = str(line.split("\t")[1]).strip()

        # try to convert to number
        # (sometimes the values are empty)
        try:
            t = float(t)
            c = float(c)
        except:
            continue

        # if values are okay, store them
        t_data.append(t)
        c_data.append(c)

    # interpolate data
    itp = PchipInterpolator(t_data, c_data)

    # save interpolated values
    t_out = np.arange(tmin, tmax, dt)
    c_out = itp(t_out)
    out = np.array([t_out,c_out]).T

    # save data to csv
    save_header = "T [K] , Cp [J mol-1 K-1]"
    outpath = os.path.join(get_gendir(), "cp", "dat")+"/%s.csv"%gas
    np.savetxt(outpath, out, header=save_header, delimiter=',', fmt="%.9e")
    return outpath

In [14]:
files = glob.glob(os.path.join(get_gendir(), "cp", "web")+"/*.txt")

nfiles = len(files)
pct_lst = -1
print("Progress (%): ", end="", flush=True)
for i,f in enumerate(files):
    # progress
    pct = int((i+1)/nfiles*100)
    if (pct%10 == 0) and (pct != pct_lst):
        pct_lst = pct
        print("%d "%pct, end="", flush=True)

    # convert
    o = convert_txt(f)

    # plot
    plot_gas(o)

print("")
print("Done")

Progress (%): 0 10 20 30 40 50 60 70 80 90 100 
Done
